# Javier Abad Hernández - Práctica 1 TAA

In [2]:
import pandas as pd
import numpy as np
from scipy.stats import entropy

from math import log2

In [3]:
# Cargar los datos
data = pd.read_csv("weather.nominal.csv")
data

,outlook,temperature,humidity,windy,play
0,sunny,hot,high,False,no
1,sunny,hot,high,True,no
2,overcast,hot,high,False,yes
3,rainy,mild,high,False,yes
4,rainy,cool,normal,False,yes
5,rainy,cool,normal,True,no
6,overcast,cool,normal,True,yes
7,sunny,mild,high,False,no
8,sunny,cool,normal,False,yes
9,rainy,mild,normal,False,yes


In [9]:
data.iloc[:, -1].value_counts(normalize=True)

yes    0.642857
no     0.357143
Name: play, dtype: float64

#### Primero lo realizo con la entropía.

In [12]:
# Calcular la entropía del conjunto de datos completo
H = entropy(data.iloc[:, -1].value_counts(normalize=True), base=2)
H

0.940285958670631

In [74]:
# Calcular la ganancia de información de cada atributo
def info_gain(data, attribute):  # La información de ganancia es la entropia, menos la entropia media de cada uno de los subconjuntos
    values, counts = np.unique(data[attribute], return_counts=True)
    subsets = [data[data[attribute] == v] for v in values]
    entropy_subsets = [entropy(subset.iloc[:, -1].value_counts(normalize=True), base=2) for subset in subsets]
    probs = counts / len(data)
    return H - np.sum(probs * entropy_subsets) # np.sum(probs * entropy_subsets) es el Resto(A) en los apuntes y H es I(D)
atn,ath=[],[]
for attributeH in data.columns[:-1]:
    print("Ganancia de información para", attributeH, ":", info_gain(data, attributeH))
    atn.append(attributeH)
    ath.append(info_gain(data, attributeH))


Ganancia de información para outlook : 0.246749819774439
Ganancia de información para temperature : 0.029222565658954758
Ganancia de información para humidity : 0.15183550136234159
Ganancia de información para windy : 0.04812703040826938


In [75]:
# encontrar la ganancia máxima en información
max_gain = 0
best_attribute = None    
for attribute in data.columns[:-1]:
    gain = info_gain(data, attribute)
    if gain > max_gain:
        max_gain = gain
        best_attribute = attribute

print("\nEl atributo con la ganancia máxima en información es:", best_attribute, "con una ganancia en información de:", max_gain,"\n")# Calcular el índice GINI de cada atributo


El atributo con la ganancia máxima en información es: outlook con una ganancia en información de: 0.246749819774439 



#### Segundo lo hago con GINI

In [85]:
def gini(data):
    Gi = []
    G_Gini = 1 - sum(data.iloc[:, -1].value_counts(normalize=True) ** 2)
    l = len(data)
    for atributo in data.columns[:-1]:
        Gi.append(G_Gini)
        for v in data[atributo].unique():
            table = data.loc[data[atributo] == v]
            Gi[-1] -= (len(table) / l) * (1 - sum(table.iloc[:, -1].value_counts(normalize=True) ** 2))
    return Gi, G_Gini

atg=[]
print("Indice GINI general: ", G_Gini)
for i in range (len(data.columns[:-1])):
    print("Índice GINI para", data.columns[i], ":", Gi[i])
    atg.append(Gi[i])
    



Indice GINI general:  0.4591836734693877
Índice GINI para outlook : 0.11632653061224485
Índice GINI para temperature : 0.018707482993197258
Índice GINI para humidity : 0.09183673469387743
Índice GINI para windy : 0.030612244897959162


In [86]:
print("Ganancia máxima en Información GINI =", data.columns[np.argmax(Gi)], "con una ganancia en información de:", np.amax(atg), "\n")

Ganancia máxima en Información GINI = outlook con una ganancia en información de: 0.11632653061224485 



#### Tabla de comparación de resultados.

In [88]:
data2 = {'ENTROPY': ath, 'GINI': atg}
df = pd.DataFrame(data2, index=[atn])
df


,ENTROPY,GINI
outlook,0.246750,0.116327
temperature,0.029223,0.018707
humidity,0.151836,0.091837
windy,0.048127,0.030612
